## Accesibilidad usando la red de calles

Se utilizó la matriz de costos que calculamos utilizando la red de calles para generar modelos de accesibilidad y compararlos con los que obtuvimos utilizando las distancias euclidianas.

Se leen los datos de oferta y demanda que ya se tenian hechos

In [1]:
import pandas as pd
import geopandas as gpd
from access import Access, weights
import contextily as ctx

/home/david/.local/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
#agebs_zmvm_centroides = gpd.read_file('datos/agebs.gpkg')
agebs_zmvm_centroides = pd.read_pickle("../data/agebs_zmvm_centroides.pkl")
escuelas = pd.read_pickle("../data/escuelas_centroides.pkl")

In [3]:
agebs_zmvm_centroides.head()

,CVEGEO,geometry,sum
0,0901000011716,POINT (2787091.708 816590.463),667384319
1,0901000012150,POINT (2793986.972 823047.548),285159163
2,0901000011133,POINT (2794967.016 819439.549),1087353
3,0901000011307,POINT (2792230.506 815397.361),1607977
4,0901000010281,POINT (2788669.707 823554.634),181012


In [4]:
escuelas

,id_escuela,geometry,area
0,0,POINT (2803465.381 815556.143),12.546194
1,1,POINT (2803882.165 824959.984),12.546194
2,2,POINT (2799924.581 832289.618),12.546194
3,3,POINT (2798477.114 830413.557),12.546194
4,4,POINT (2781928.380 819805.051),12.546194
...,...,...,...
1489,2377,POINT (2809662.172 824784.191),12.546194
1490,2379,POINT (2803013.636 832721.858),12.546194
1491,2380,POINT (2794446.362 812812.916),12.546194
1492,2381,POINT (2794056.001 835133.949),12.546194


Ahora leemos la matriz de costos

In [ ]:
costos = pd.read_pickle("../data/matriz_escuelas_od_walking.pkl")
costos.head()

Instanciamos el objeto `access`

In [ ]:
accesibilidad_red = Access(demand_df            = agebs_zmvm_centroides,
                           demand_index         = 'CVEGEO',
                           demand_value         = 'POBTOT',
                           supply_df            = escuelas,
                           supply_index         = 'id_escuela',
                           supply_value         = 'area',
                           cost_df              = costos,
                           cost_origin          = 'origen',
                           cost_dest            = 'destino',
                           cost_name            = 'costo',
                           neighbor_cost_df     = costos,
                           neighbor_cost_origin = 'origen',
                           neighbor_cost_dest   = 'destino',
                           neighbor_cost_name   = 'costo')

Definimos la función de peso para el modelo gravitatorio

In [ ]:
gravity = weights.gravity(scale = 750, alpha = -1)

Calculamos los mismos dos modelos de accesibilidad

In [ ]:
accesibilidad_red.weighted_catchment(name="gravity", weight_fn=gravity)
accesibilidad_red.raam(name="raam", tau=750)

Recordemos ráído lo que nos regresan los cálculos

In [ ]:
accesibilidad_red.norm_access_df

Realizamos mapas comparativos

In [ ]:
mapa_accesibilidad = agebs_zmvm_centroides.set_index('CVEGEO')[['geometry']].join(accesibilidad_red.norm_access_df, how = "inner")
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(32,12))
ax1 = mapa_accesibilidad.to_crs(epsg=3857).plot('raam_area', legend = True,
                                                cmap =  "viridis_r", 
                                                markersize = 5, alpha = 0.9, ax = ax1,
                                                vmin = mapa_accesibilidad['raam_area'].quantile(0.05), 
                                                vmax = mapa_accesibilidad['raam_area'].quantile(0.95),
                                                )
ax1.set_axis_off()
ax1.set(title='Modelo RAAM')
ctx.add_basemap(ax1, source=ctx.providers.CartoDB.Positron)

ax2 = mapa_accesibilidad.to_crs(epsg=3857).plot('gravity_area', legend = True,
                                                cmap =  "viridis", 
                                                markersize = 5, alpha = 0.9, ax = ax2,
                                                vmin = mapa_accesibilidad['gravity_area'].quantile(0.05), 
                                                vmax = mapa_accesibilidad['gravity_area'].quantile(0.95),
                                                )
ax2.set_axis_off()
ax2.set(title='Modelo Gravitatorio')
# plt.subplots_adjust(wspace=-.5, hspace=0)
ctx.add_basemap(ax2, source=ctx.providers.CartoDB.Positron)
plt.tight_layout()

## Comparación con distancia euclidiana

Comparamos los modelos que calculamos usando la distancia euclidiana con los que obtuvimos ahora. Los resultados anteriores están guardados como csv en `datos/accesibilidad_distancia_euclidiana.csv`.

In [ ]:
accesibilidad_euclidiana = pd.read_csv("datos/accesibilidad_distancia_euclidiana.csv")
accesibilidad_euclidiana

Lo primero que podemos hacer es unos mapas para comparar visualmente los resultados. Para esto lo mejor es hacer una función que haga cada mapa

In [ ]:
def haz_mapa(datos, ax, columna, titulo, swap_colors=True):
    if swap_colors:
        swap_colors = "raam" in columna
    ax = datos.to_crs(epsg=3857).plot(columna, 
                                      legend = True,
                                      cmap = "viridis_r" if swap_colors else "viridis", 
                                      markersize = 5, alpha = 0.9, ax = ax,
                                      vmin = datos[columna].quantile(0.05), 
                                      vmax = datos[columna].quantile(0.95),
                                      )
    ax.set_axis_off()
    ax.set(title=titulo)
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)

Con esta función es fácil hacer todos los mapas en una misma figura, pero primero necesitamos tener los resultados originales ligados a las geometrías de los puntos. Lo más fácil es tener todo en un sólo dataframe, claro que eso implica cambiar los nombres de las columnas

In [ ]:
mapa_accesibilidad = mapa_accesibilidad.rename({'gravity_area': 'gravity_area_red', 'raam_area': 'raam_area_red'}, axis=1)
mapa_accesibilidad = mapa_accesibilidad.merge(accesibilidad_euclidiana, on='CVEGEO')
mapa_accesibilidad = mapa_accesibilidad.rename({'gravity_area': 'gravity_area_euclidiana', 'raam_area': 'raam_area_euclidiana'}, axis=1)
mapa_accesibilidad

Ahora sí podemos hacer todos los mapas

In [ ]:
titulos_columnas = {'gravity_area_euclidiana': 'Gravitatorio Distancia Euclidiana',
                    'raam_area_euclidiana': 'RAAM Distancia Euclidiana',
                    'gravity_area_red': 'Gravitatorio Distancia sobre la Red',
                    'raam_area_red': 'RAAM Distancia sobre la Red'}
fig, axes = plt.subplots(2,2,figsize=(30,24))
axes = axes.ravel()
for i, columna in enumerate(titulos_columnas.keys()):
    haz_mapa(mapa_accesibilidad, axes[i], columna, titulos_columnas[columna])

Son muy parecidos, pero no exáctamente iguales, podemos calcular las diferencias en los valores de accesibilidad por modelo y hacer mapas de las diferencias

In [ ]:
mapa_accesibilidad['dif_gravity'] = mapa_accesibilidad['gravity_area_euclidiana'] - mapa_accesibilidad['gravity_area_red']
mapa_accesibilidad['dif_raam'] = mapa_accesibilidad['raam_area_euclidiana'] - mapa_accesibilidad['raam_area_red']
mapa_accesibilidad

In [ ]:
titulos_columnas = {'dif_gravity': 'Diferencia en modelos gravitatorios',
                    'dif_raam': 'Diferencia en modelos RAAM'
                   }
fig, axes = plt.subplots(1,2,figsize=(30,24))
axes = axes.ravel()
for i, columna in enumerate(titulos_columnas.keys()):
    haz_mapa(mapa_accesibilidad, axes[i], columna, titulos_columnas[columna], swap_colors=False)